# Predict Future Sales

You are provided with daily historical sales data. The task is to forecast the total amount of products sold in every shop for the test set. Note that the list of shops and products slightly changes every month. Creating a robust model that can handle such situations is part of the challenge.

###### Goal: 
item_cnt_day - number of products sold. You are predicting a monthly amount of this measure

##### File descriptions
    sales_train.csv - the training set. Daily historical data from January 2013 to October 2015.
    test.csv - the test set. You need to forecast the sales for these shops and products for November 2015.
    
    items.csv - supplemental information about the items/products.
    item_categories.csv  - supplemental information about the items categories.
    shops.csv- supplemental information about the shops.

### 1.0 Read data

In [24]:
import pandas as pd

#!pip install googletrans
from googletrans import Translator
# translate name to english
translator = Translator()

#!pip install plotly
import plotly
plotly.tools.set_credentials_file(username='', api_key='')

import plotly.plotly as py
import plotly.graph_objs as go

In [2]:
sales_train=pd.read_csv('sales_train_v2.csv')
sales_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [3]:
# change date_block_num to month, year column
date_num = sales_train.date.str.split(".",n=2,expand=True)
date_num.columns = ['Date','Month','Year']
sales_train_result = pd.concat([sales_train, date_num], axis=1)
sales_train_result.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,Date,Month,Year
0,02.01.2013,0,59,22154,999.00,1.0,02,01,2013
1,03.01.2013,0,25,2552,899.00,1.0,03,01,2013
2,05.01.2013,0,25,2552,899.00,-1.0,05,01,2013
3,06.01.2013,0,25,2554,1709.05,1.0,06,01,2013
4,15.01.2013,0,25,2555,1099.00,1.0,15,01,2013


In [4]:
test=pd.read_csv('test.csv')
test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [5]:
items=pd.read_csv('items.csv')
items.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [6]:
shops=pd.read_csv('shops.csv')
shops.head()

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


In [7]:
item_categories=pd.read_csv('item_categories.csv')
item_categories.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [8]:
# reference: https://www.codeproject.com/Tips/1236705/How-to-Use-Google-Translator-in-Python

item_categories['item_category_name_ENG'] = 0
for i in range (len(item_categories['item_category_name'])):
    result = translator.translate(item_categories['item_category_name'][i])
    item_categories['item_category_name_ENG'][i] = result.text
item_categories.head()

C:\Users\Claire\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Claire\Anaconda3\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,item_category_name,item_category_id,item_category_name_ENG
0,PC - Гарнитуры/Наушники,0,PC - Headsets / Headphones
1,Аксессуары - PS2,1,Accessories - PS2
2,Аксессуары - PS3,2,Accessories - PS3
3,Аксессуары - PS4,3,Accessories - PS4
4,Аксессуары - PSP,4,Accessories - PSP


In [9]:
# split to main category and subcategory
cate = item_categories.item_category_name_ENG.str.split("-",n=1,expand=True)
cate.columns = ['category1','category2']
item_categories_result = pd.concat([item_categories, cate], axis=1)
item_categories_result.head()

,item_category_name,item_category_id,item_category_name_ENG,category1,category2
0,PC - Гарнитуры/Наушники,0,PC - Headsets / Headphones,PC,Headsets / Headphones
1,Аксессуары - PS2,1,Accessories - PS2,Accessories,PS2
2,Аксессуары - PS3,2,Accessories - PS3,Accessories,PS3
3,Аксессуары - PS4,3,Accessories - PS4,Accessories,PS4
4,Аксессуары - PSP,4,Accessories - PSP,Accessories,PSP


### 2.0 Data Cleaning 

Relational Data Connection to 1 data frame for training

In [10]:
df = pd.merge(sales_train_result, items, how='left', on='item_id')
df2 = pd.merge(df, shops, how='left', on='shop_id')
df3 = pd.merge(df2, item_categories_result, how='left', on='item_category_id')

In [11]:
## Select features that will be used for prediction

#df_train =df3[['date','date_block_num','shop_id','shop_name','item_category_id','item_category_name','item_id','item_name','item_price','item_cnt_day']]
df_train =df3[['date_block_num','shop_id','item_category_id','category1','category2','item_id','item_price','item_cnt_day']]
df_train.head()

,date_block_num,shop_id,item_category_id,category1,category2,item_id,item_price,item_cnt_day
0,0,59,37,Cinema,Blu-Ray,22154,999.00,1.0
1,0,25,58,Music,Vinyl,2552,899.00,1.0
2,0,25,58,Music,Vinyl,2552,899.00,-1.0
3,0,25,58,Music,Vinyl,2554,1709.05,1.0
4,0,25,56,Music,CD of branded production,2555,1099.00,1.0


In [12]:
# general data description before prediction

print('Shape of the dataframe:', df_train.shape)
print("The general data of train:")
df_train.describe()

Shape of the dataframe: (2935849, 8)
The general data of train:


,date_block_num,shop_id,item_category_id,item_id,item_price,item_cnt_day
count,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06
mean,1.456991e+01,3.300173e+01,4.000138e+01,1.019723e+04,8.908532e+02,1.242641e+00
std,9.422988e+00,1.622697e+01,1.710076e+01,6.324297e+03,1.729800e+03,2.618834e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,-2.200000e+01
25%,7.000000e+00,2.200000e+01,2.800000e+01,4.476000e+03,2.490000e+02,1.000000e+00
50%,1.400000e+01,3.100000e+01,4.000000e+01,9.343000e+03,3.990000e+02,1.000000e+00
75%,2.300000e+01,4.700000e+01,5.500000e+01,1.568400e+04,9.990000e+02,1.000000e+00
max,3.300000e+01,5.900000e+01,8.300000e+01,2.216900e+04,3.079800e+05,2.169000e+03


### 3.0 Data Visulization

Product count plot by shop, category(main&sub); product count relation  


In [25]:
# Main category

x = df_train['category1'].value_counts().index.values.astype('str')
y = df_train['item_cnt_day'].value_counts().values
pct = [("%.2f"%(v*100))+"%"for v in (y/len(df_train))]

In [26]:
trace1 = go.Bar(x=x, y=y, text=pct)
layout = dict(title= 'Number of Items by Main Category',
              yaxis = dict(title='Count'),
              xaxis = dict(title='Category'))
fig=dict(data=[trace1], layout=layout)
py.iplot(fig)

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~clairegao001/0 or inside your plot.ly account where it is named 'plot from API'


In [27]:
# Sub category

x = df_train['category2'].value_counts().index.values.astype('str')
y = df_train['item_cnt_day'].value_counts().values
pct = [("%.2f"%(v*100))+"%"for v in (y/len(df_train))]

In [28]:
trace1 = go.Bar(x=x, y=y, text=pct)
layout = dict(title= 'Number of Items by Sub Category',
              yaxis = dict(title='Count'),
              xaxis = dict(title='Category'))
fig=dict(data=[trace1], layout=layout)
py.iplot(fig)

In [29]:
# Shop

x = df_train['shop_id'].value_counts().index.values
y = df_train['item_cnt_day'].value_counts().values
pct = [("%.2f"%(v*100))+"%"for v in (y/len(df_train))]

In [30]:
trace1 = go.Bar(x=x, y=y, text=pct)
layout = dict(title= 'Number of Items by Shop',
              yaxis = dict(title='Count'),
              xaxis = dict(title='Category'))
fig=dict(data=[trace1], layout=layout)
py.iplot(fig)

### 4.0 Model building

In [ ]:
# use train/test split with different random_state values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print("X_train Shape: {}".format(X_train.shape), "\ny_train Shape: {}".format(y_train.shape),
      "\nX_test Shape: {}".format(X_test.shape), "\ny_test Shape: {}".format(y_test.shape))